In [30]:
import numpy as np 
import matplotlib.pyplot as plt
import gym
import ray.rllib.agents.ppo as ppo
import random
import jdc
import pandas as pd

from gym import Env, spaces
import time

#get a matrix of distances between locations
df = pd.read_csv('dist_vologda_matrix.csv', sep=',')
df = df.iloc[:, 1:]

N = len(df.values[0])
df = df / 1000
df


,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,99.2272,101.8513,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
1,99.0684,0.0000,4.9465,134.5201,149.5589,748.0318,5.5280,491.2703,488.6546,241.7276,...,581.1570,581.0711,578.0967,579.8263,581.8535,581.8288,575.0368,576.5238,473.7510,448.7008
2,101.8148,5.0688,0.0000,137.2665,152.3053,750.7781,3.0302,494.0166,491.4010,244.4739,...,583.9033,583.8174,580.8430,582.5726,584.5999,584.5752,577.7831,579.2702,476.4973,451.4472
3,139.2135,134.6791,137.3031,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,149.7178,152.3419,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,582.6174,585.2414,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,576.6275,579.2515,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,577.9779,580.6020,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,475.4954,478.1194,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [40]:
for i in range(len(df.values[0])):
    if 2 * df.values[0, i] / 60 < 12:
        df.drop(df.columns[i], axis=1)
        df.drop(df.index[i])
df

,АЗС №35400 Кичменгский Городок,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,АЗС №35410 Сазоново-1,АЗС №35411 Сазоново-2,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,782.7022,782.2145,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
3,139.2135,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,619.0077,618.5201,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,619.3335,618.8458,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
5,778.9885,615.1980,615.5238,0.0000,753.0253,349.0547,346.4390,512.4135,142.9597,142.4720,...,187.4480,187.3622,184.7850,188.3816,183.6881,191.1772,185.1426,186.6296,274.4141,309.3918
6,102.4329,137.8846,152.9234,751.3963,0.0000,494.6347,492.0191,245.0921,756.8350,756.3473,...,584.5214,584.4356,581.4611,583.1907,585.2180,585.1933,578.4012,579.8883,477.1154,452.0653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,196.6738,196.1861,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,190.5069,190.0192,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,191.8574,191.3697,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,279.8093,279.3216,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [2]:
#custom gym environment for our problem
class TransportScape(Env):
    def __init__(self, env_config):
        self.num_of_trucks = N - 1
        self.num_of_locations = N - 1
        self.working_hours = 12 #working hours of each truck driver (harsh reality)
        self.truck_mean_speed = 60
        self.distances = df.values #an array, containing distances between locations

#         self.action_space = spaces.Tuple((
#             #i - truck number
#             spaces.Discrete(self.num_of_trucks),
#             #j - location number
#             spaces.Discrete(self.num_of_locations)
#         ))
        self.action_space = spaces.MultiDiscrete([self.num_of_trucks, self.num_of_locations])
    
        self.observation_space = spaces.Dict(
        {
            #x_i - shows each truck's location
            #MultiDiscrete(np.full(self.num_of_trucks, self.num_of_locations))
            'truck location': spaces.Box(low=np.zeros(self.num_of_trucks), high=np.full(self.num_of_trucks, self.num_of_locations), dtype=np.int64),
            #p_j - shows whether a truck is assigned to this particular location or not
            'assignment': spaces.Box(low=np.zeros(self.num_of_locations), high=np.full(self.num_of_locations, 1), dtype=np.int64), 
            #u_i - shows whether a particular truck was used before
            'truck usage': spaces.Box(low=np.zeros(self.num_of_trucks), high=np.full(self.num_of_trucks, 1), dtype=np.int64),
            #delta_i - shows how much time each truck has left until the end of the work day
            'time left': spaces.Box(low=np.zeros(self.num_of_trucks), high=np.full(self.num_of_trucks, self.working_hours), dtype=np.float64)
        })
        self.reset()
        
    def reset(self):
        self.state = {'truck location': np.full(self.num_of_trucks, self.num_of_locations),
                      'assignment': np.zeros(self.num_of_locations),
                      'truck usage': np.zeros(self.num_of_trucks),
                      'time left': np.full(self.num_of_trucks, self.working_hours)}
        self.reward = 0
        self.done = False
        return self.state 
    
    def step(self, action):
        i, j = action[0], action[1]
        
        truck_loc = self.state['truck location']
        assignment = self.state['assignment']
        truck_use = self.state['truck usage']
        time_left = self.state['time left']
        
        #if all locations have trucks assigned to them - end episode
        if np.all(self.state['assignment'] == 1):
            self.done = True
        #if a truck is already assigned to this location - penalty
        if assignment[j] == 1:
            self.reward -= 100
        #if no time left - penalty
        if (time_left[i] - self.distances[truck_loc[i], j] / self.truck_mean_speed - self.distances[truck_loc[-1], j] / self.truck_mean_speed) < 0:
            self.reward -= 100
        #else - assign truck to a new location
        else:
            #if needs to use an unused before truck - penalty
            if truck_use[i] == 0:
                self.reward -= 100
                truck_use[i] = 1
            assignment[i] = 1
            self.reward -= self.distances[truck_loc[i], j]
            time_left[i] -= self.distances[truck_loc[i], j] / self.truck_mean_speed
            truck_loc[i] = j
        
        self.state['truck location'] = truck_loc
        self.state['assignment'] = assignment
        self.state['truck usage'] = truck_use
        self.state['time left'] = time_left
        
        return self.state, self.reward, self.done, {}

In [3]:
config = ppo.DEFAULT_CONFIG.copy()
config["env_config"] = {}
config["num_workers"] = 1
config["framework"] = "torch"
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
#config["disable_env_checking"] = True
#config

In [41]:
trainer = ppo.PPOTrainer(config, TransportScape)

(RolloutWorker pid=6017) 2022-04-27 18:50:53,068	WARNING rollout_worker.py:1723 -- Validating sub-env at vector index=0 ... (NOT OK)
(RolloutWorker pid=6017) 2022-04-27 18:50:53,077	ERROR worker.py:430 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=6017, ip=192.168.1.72, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f0d9c7fcc40>)
(RolloutWorker pid=6017)   File "/home/basilshim/anaconda3/envs/deeplom/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 462, in __init__
(RolloutWorker pid=6017)     _validate_env(self.env, env_context=self.env_context)
(RolloutWorker pid=6017)   File "/home/basilshim/anaconda3/envs/deeplom/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1724, in _validate_env
(RolloutWorker pid=6017)     raise EnvError(
(RolloutWorker pid=6017) ray.rllib.utils.error.EnvError: Env's `observation_space` Dict(assignm

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=6017, ip=192.168.1.72, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f0d9c7fcc40>)
  File "/home/basilshim/anaconda3/envs/deeplom/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 462, in __init__
    _validate_env(self.env, env_context=self.env_context)
  File "/home/basilshim/anaconda3/envs/deeplom/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1724, in _validate_env
    raise EnvError(
ray.rllib.utils.error.EnvError: Env's `observation_space` Dict(assignment:Box([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], (63,), int64), time left:Box([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [6]:
from stable_baselines3 import PPO

env = TransportScape(config)
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 366       |
|    ep_rew_mean     | -1.44e+07 |
| time/              |           |
|    fps             | 1082      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------


In [42]:
obs = env.reset()
result = []
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    print(i)
    print(action)
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()
        break
env.close()

61
[40 46]
61
[56 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

...

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]
61
[40 46]

KeyboardInterrupt: 

In [ ]:
#plot reward max